## Assemblage du jeu de données pour les modèles uniclasse & multilabel

#### 1. Assemblage des achats réels et shortlists 
    - Données d'entraînement / validation : élaboration de la shortlist sur les semaines [1, 56], liste d'achat sur la semaine 0.
    - Données de test : élaboration de la shortlist sur les semaines [0, 54].
#### 2. Création d'un dataset pour entraînement uniclasse.
#### 3. Création d'un dataset pour entraînement multilabels.

### Assemblage des achats réels et shortlists

In [2]:
import pandas as pd
import numpy as np

ages_centers = [15, 25, 35, 45, 55, 65]

In [3]:
transactions = pd.read_pickle('pickles/transactions.pkl')

In [4]:
customers = pd.read_pickle('pickles/customers_second_iteration.pkl')
articles = pd.read_pickle('pickles/articles_second_iteration.pkl')

In [5]:


def top_sales_cross_lists(week_sales):
    """Création des listings nécessaires à l'élaboration de la liste croisée âge / catégorie / groupe.

    Args:
        week_sales (pd.DataFrame): Les transactions pour la semaine choisie

    Returns:
        dictionnary: Listes par groupe et par catégorie, avec les pondérations par âge.
    """
    global articles
    global customers
    
    top_sales = week_sales.merge(customers, on = 'customer_id', how = 'left')
    
    top_sales = top_sales.groupby(['article_id', 'group'], as_index = False).agg(
        sales = ('article_id', 'count'),
        age_around_15 = ('age_around_15', 'sum'),
        age_around_25 = ('age_around_25', 'sum'),
        age_around_35 = ('age_around_35', 'sum'),
        age_around_45 = ('age_around_45', 'sum'),
        age_around_55 = ('age_around_55', 'sum'),
        age_around_65 = ('age_around_65', 'sum'),
    )
    
    top_sales = top_sales.merge(articles[['article_id', 'index_group_name']], on = 'article_id')
    
    top_sales['index_group_name'] = top_sales['index_group_name'].apply(lambda x: x.split('/')[0].lower())
    
    # Préparation des listes.

    groups = top_sales['group'].unique()
    categories = top_sales['index_group_name'].unique()

    cross_lists = {}

    for group in groups :
        cross_lists[group] = {}
        
        categories = top_sales['index_group_name'].unique()

        for category in categories:
            
            cross_lists[group][category] = top_sales[(top_sales['group'] == group) & (top_sales['index_group_name'] == category)]
            cross_lists[group][category]['shares'] = cross_lists[group][category]['sales'] / cross_lists[group][category]['sales'].sum()
            cross_lists[group][category]['index_group_name'] = category
            cross_lists[group][category] = cross_lists[group][category].nlargest(n = 100, columns = 'shares')
        
    
    return cross_lists

def get_cross_list(x):
    global cross_lists
    
    top_sales_weighted = []
    group = x['group']
    
    for category in cross_lists[group].keys():
    
        if x[category] == 0:
            continue
        
        cross_lists[group][category]['shares_weighted'] = cross_lists[group][category]['shares'] * x[category]
        cross_lists[group][category]['sales_cross_weighted'] = 0
            
        for age in ages_centers:
            column = f"age_around_{age}"
            
            if x[column] == 0:
                continue
            
            cross_lists[group][category]['sales_cross_weighted'] += cross_lists[group][category][column] * cross_lists[group][category]['shares_weighted']
            
            top_sales_weighted.append(cross_lists[group][category])
    
    if len(top_sales_weighted) == 0:
        return []
    
    final_list = pd.concat(top_sales_weighted).groupby('article_id', as_index = False).agg(
        sales_cross_weighted = ('sales_cross_weighted', 'max')
    )
    
    return final_list.nlargest(n = 50, columns = ['sales_cross_weighted'])['article_id'].tolist()

def process_shortlist(raw, shortlist_length = 50, repurchase_amount = 8, pair_amount = 29):
    shortlist = []
    
    i_1 = 0
    i_2 = 0
    i_3 = 0
    
    
    while (len(shortlist) < shortlist_length) & (i_3 < len(raw['cross_list'])):
        if i_1 < repurchase_amount and i_1 < len(raw['repurchase_list']):
            article_id = raw['repurchase_list'][i_1]
            i_1 += 1
        elif i_1 + i_2 < repurchase_amount + pair_amount and i_2 < len(raw['pair_list']):
            article_id = raw['pair_list'][i_2]
            i_2 += 1
        else:
            article_id = raw['cross_list'][i_3]
            i_3 += 1
    
        if article_id not in shortlist:
            shortlist.append(article_id)
            
            
    return shortlist

In [9]:
# Ajout des features Client nécessaires à l'élaboration des listes.
customers_cross_data = customers.drop(['FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code_group', 'average_cart_articles',
       'average_cart_price', 'total_carts', 'total_articles', 'total_price',
       'average_cart_interval', 'repurchases', 'repurchases_interval'], 1)

/tmp/ipykernel_5261/3900466938.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  customers_cross_data = customers.drop(['FN', 'Active', 'club_member_status',


### Données d'entraînement
#### Liste des achats effectués

In [5]:
# Liste des achats effectués

data_train = transactions[transactions['week_number'] == 0].groupby('customer_id', as_index = False).agg(
    purchased_list = ('article_id', lambda x: list(x))
)

In [7]:
data_train = data_train.merge(customers_cross_data, on = 'customer_id', how = 'left')

#### Liste croisée groupe / catégorie / âge

In [8]:
cross_lists = top_sales_cross_lists(transactions[transactions['week_number'] == 1])
data_train['cross_list'] = data_train.apply(lambda x: get_cross_list(x), axis = 1)

/tmp/ipykernel_5634/2798358702.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cross_lists[group][category]['shares'] = cross_lists[group][category]['sales'] / cross_lists[group][category]['sales'].sum()
/tmp/ipykernel_5634/2798358702.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cross_lists[group][category]['index_group_name'] = category


In [9]:
### CHECKPOINT
data_train.to_pickle('pickles/second_iteration_data_train.pkl')

#### Articles appairés

In [10]:
pairs = pd.read_pickle('pickles/article_pairs.pkl')

In [25]:
transactions_train = transactions[transactions['week_number'] <= 56]

transaction_train = transactions_train.merge(
    data_train[['customer_id']], 
    on = 'customer_id',
    how = 'inner'
)

In [13]:
transactions_to_pair = transaction_train.groupby(['customer_id', 'article_id'], as_index = False).agg(
    week_number = ('week_number', 'min')
)

In [ ]:
transactions_to_pair['article_id'] = transactions_to_pair['article_id'].astype('int32')

In [15]:
# Détermination des paires.
pairs = transactions_to_pair.merge(pairs, on='article_id', how='inner')

In [16]:
# Suppression des lignes où l'article appairé a déjà été acheté.
pairs = pairs.merge(transactions_to_pair, left_on=['customer_id', 'article_dest'], right_on=['customer_id', 'article_id'], how = 'left', suffixes = ('_source', '_dest'))

pairs['week_number_dest'].fillna(100, inplace = True)


pairs = pairs[pairs['week_number_source'] > pairs['week_number_dest']]
pairs['article_dest'] = '0' + pairs['article_dest'].astype(str)

In [17]:
# Finalisation des listes.
pairs_list = pairs.groupby('customer_id', as_index = False, sort = False).agg(
    list = ('article_dest', lambda x: list(x))
)
data_train['pair_list'] = pairs_list['list'].apply(lambda x: x[0:51])

In [ ]:
### CHECKPOINT
data_train.to_pickle('pickles/second_iteration_data_train.pkl')

#### Rachats d'articles

In [20]:
customers_repurchases = customers[['customer_id', 'repurchases', 'repurchases_interval']]
articles_repurchases = articles[['article_id', 'repurchases', 'repurchase_interval']]

In [26]:
already_purchased = transaction_train.groupby(['customer_id', 'article_id'], as_index = False).agg(
    last_week = ('week_number', 'min'),
    week_list = ('week_number', lambda x: list(x)),
    count = ('article_id', 'count')
)

already_purchased = already_purchased[already_purchased['count'] > 1]

In [27]:
already_purchased = already_purchased.merge(customers_repurchases, on = 'customer_id', how = 'left')
already_purchased = already_purchased.merge(articles_repurchases, on = 'article_id', how = 'left', suffixes = ('_customer', '_article'))

In [28]:
intervals_distribution = pd.read_pickle('pickles/repurchases_interval_distribution.pkl')

In [29]:
# Calcul du score en fonction des données de l'article, du client, et du temps depuis le dernier achat.

already_purchased['mean_interval'] = (already_purchased['repurchases_interval'] + already_purchased['repurchase_interval']) / 2
already_purchased['score'] = already_purchased['repurchases_article'] * already_purchased['repurchases_customer']
already_purchased['interval_weighted'] =  already_purchased['last_week'] / already_purchased['mean_interval']
already_purchased = already_purchased[already_purchased['interval_weighted'] <= 50]
already_purchased['score'] *= already_purchased['interval_weighted'].apply(lambda x: 
    intervals_distribution.loc[round(x)]
)

already_purchased = already_purchased[['customer_id', 'article_id', 'score']].sort_values(['customer_id', 'score'], ascending = False)

already_purchased.reset_index(drop = True, inplace = True)

In [31]:
repurchase_lists = already_purchased.groupby('customer_id', as_index = False, sort = False).agg(
    list = ('article_id', lambda x: list(x))
)

In [32]:
data_train['repurchase_list'] = repurchase_lists['list']

In [35]:
### CHECKPOINT
data_train.to_pickle('pickles/second_iteration_data_train.pkl')

### Fusion des listes 
Selon les proportions déterminées dans `articles_shortlist.ipynb`

In [3]:
import pandas as pd

data_train = pd.read_pickle('pickles/second_iteration_data_train.pkl')

In [7]:
# Nettoyage
data_train = data_train.fillna('').apply(list)

In [9]:
data_train['shortlist'] = data_train.apply(lambda x: process_shortlist(x), axis = 1)

In [12]:
data_train = data_train[['customer_id', 'purchased_list', 'cross_list', 'pair_list', 'repurchase_list', 'shortlist']]

In [14]:
data_train.to_pickle('pickles/second_iteration_data_train_lists.pkl')
data_train.head(8)

,customer_id,purchased_list,cross_list,pair_list,repurchase_list,shortlist
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[0624486001],"[0915526001, 0915526001, 0919365008, 091936500...",[0751471043],"[0748269009, 0881919001, 0895610005, 090222900...","[0748269009, 0881919001, 0895610005, 090222900..."
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[0827487003],"[0919365008, 0919365008, 0706016001, 070601600...","[0751471043, 0554772003, 0858052008, 083211400...","[0652924004, 0797892001, 0900176002, 044851500...","[0652924004, 0797892001, 0900176002, 044851500..."
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[0757926001, 0788575004, 0640021019]","[0706016001, 0706016001, 0448509014, 044850901...","[0862482007, 0875350003, 0832453003, 082793500...","[0111593001, 0658030020, 0863001001]","[0111593001, 0658030020, 0863001001, 086248200..."
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[0874110016],"[0919365008, 0706016001, 0797988002, 044850901...",[0751471043],"[0714790020, 0448509018, 0658298001, 079479300...","[0714790020, 0448509018, 0658298001, 079479300..."
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[0903762001, 0879189005, 0799365027, 048663900...","[0915526001, 0915526001, 0919365008, 091936500...","[0751471043, 0751471043, 0562245050, 087253700...","[0687704022, 0759465001, 0887949001, 085912500...","[0687704022, 0759465001, 0887949001, 085912500..."
5,0008daf19b2a7cd6fa00836c717aa0f143c83d88c35e02...,[0786304008],"[0915526001, 0915526001, 0751471001, 075147100...","[0751471043, 0695632107, 0762846031, 088946000...","[0891322004, 0797603001, 0670698003, 091936500...","[0891322004, 0797603001, 0670698003, 091936500..."
6,000fb6e772c5d0023892065e659963da90b1866035558e...,"[0889669006, 0913272003, 0786022008, 091327200...","[0751471043, 0751471043, 0751471001, 075147100...","[0862482007, 0877274003, 0875469002, 0781225001]","[0655784001, 0805011013, 0783497006, 0640539013]","[0655784001, 0805011013, 0783497006, 064053901..."
7,0010e8eb18f131e724d6997909af0808adbba057529edb...,"[0610776105, 0562245001, 0237347060, 045616306...","[0915526001, 0898694001, 0751471001, 091552900...","[0862482007, 0599580038, 0885072001, 084260500...",[0923134005],"[0923134005, 0862482007, 0599580038, 088507200..."


### Données de test
Utilisées pour compiler la prédiction finale.

In [5]:
# Liste compilée dans `already_purchased_items.ipynb`
repurchase_lists = pd.read_pickle('pickles/already_purchased_list.pkl')

# Liste compilée dans `items_purchased_together.ipynb`
pair_lists = pd.read_pickle('pickles/articles_pairs_list.pkl')

In [6]:
data_test = customers[['customer_id']]
data_test = data_test.merge(repurchase_lists, on = 'customer_id', how = 'left')
data_test.rename({'list': 'repurchase_list'}, axis = 1, inplace = True)

In [7]:
data_test = data_test.merge(pair_lists, on = 'customer_id', how = 'left')
data_test.rename({'list': 'pair_list'}, axis = 1, inplace = True)

In [10]:
data_test = data_test.merge(customers_cross_data, on = 'customer_id', how = 'left')

In [ ]:
cross_lists = top_sales_cross_lists(transactions[transactions['week_number'] == 0])

In [12]:
# Génération de la cross-list par blocs de 50 000 clients.
length = len(data_test)
chunks = []
index = 0
rows_by_block = 50000

while index < length:
    print(f"\r Traitement des lignes {index}-{index + rows_by_block}       ", end = "")
    end = min(index * rows_by_block, length)

    chunk = data_test.loc[index : min(index + rows_by_block, length)].apply(lambda x: get_cross_list(x), axis = 1)
    
    chunks.append(chunk)
    
    index += rows_by_block

 Traitement des lignes 1350000-1400000       

In [13]:
cross_list = pd.concat(chunks)

In [14]:
cross_list.to_pickle('pickles/cross_list.pkl')

In [22]:
data_test = pd.concat([data_test, cross_list.rename('cross_list')], axis = 1)

In [23]:
data_test = data_test[['customer_id', 'cross_list', 'pair_list', 'repurchase_list']]

In [28]:
data_test = data_test.fillna('').apply(list)

In [32]:
data_test['shortlist'] = data_test.apply(lambda x: process_shortlist(x), axis = 1)

In [3]:
data_test = pd.read_pickle('pickles/second_iteration_data_test_lists.pkl')

In [9]:
data_test['submission'] = data_test.apply(lambda x: process_shortlist(x, 12, 0, 12), axis = 1)

In [35]:
data_test.to_pickle('pickles/second_iteration_data_test_lists.pkl')
data_test.head(8)

,customer_id,cross_list,pair_list,repurchase_list,shortlist,submission
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0751471001, 0915529003, 0928206001, 057308504...",,,"[0751471001, 0915529003, 0928206001, 057308504...","[0751471001, 0915529003, 0928206001, 057308504..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0924243001, 0918522001, 0915529005, 092424300...",,"[0590928022, 0599580049, 0800436010]","[0590928022, 0599580049, 0800436010, 092424300...","[0590928022, 0599580049, 0800436010, 092424300..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0918522001, 0924243001, 0866731001, 091829200...",,,"[0918522001, 0924243001, 0866731001, 091829200...","[0918522001, 0924243001, 0866731001, 091829200..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0866731001, 0852584001, 0791587001, 073068306...",,,"[0866731001, 0852584001, 0791587001, 073068306...","[0866731001, 0852584001, 0791587001, 073068306..."
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0924243001, 0930380001, 0924243002, 092820600...",,[0399061015],"[0399061015, 0924243001, 0930380001, 092424300...","[0399061015, 0924243001, 0930380001, 092424300..."
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,"[0918292001, 0866731001, 0869331006, 085258400...",,,"[0918292001, 0866731001, 0869331006, 085258400...","[0918292001, 0866731001, 0869331006, 085258400..."
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,"[0911699002, 0788575004, 0874110016, 091103400...",,,"[0911699002, 0788575004, 0874110016, 091103400...","[0911699002, 0788575004, 0874110016, 091103400..."
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,"[0924243001, 0909370001, 0923758001, 093554100...",,,"[0924243001, 0909370001, 0923758001, 093554100...","[0924243001, 0909370001, 0923758001, 093554100..."


#### Soumission de 12 articles issus de la shortlist.

In [7]:
data_test.drop_duplicates(subset='customer_id', inplace = True)

In [8]:
submission = data_test[['customer_id']]
submission['prediction'] = data_test['submission'].apply(lambda x: " ".join(x))
submission.to_csv('submissions/submission_crosslist.csv', index = False)

/tmp/ipykernel_18598/1054230105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['prediction'] = data_test['submission'].apply(lambda x: " ".join(x))


### Création du Dataset pour entraînement uniclasse.
Les features catégorielles étant laissées telles quelles pour le moment.

In [2]:
import pandas as pd
data_train = pd.read_pickle('pickles/second_iteration_data_train_lists.pkl')

In [45]:
from utils.list_to_uniclass import ListToUniclass

list_to_uniclass = ListToUniclass()

data_train_uniclass = list_to_uniclass.fit_transform(data_train)

 Création DataFrame uniclasse 68984 / 68984

In [46]:
### Checkpoint
data_train_uniclass.to_pickle('pickles/second_iteration_data_train_uniclass.pkl')

In [3]:
data_train_uniclass = pd.read_pickle('pickles/second_iteration_data_train_uniclass.pkl')

In [4]:
articles = pd.read_pickle('pickles/articles_second_iteration.pkl')
customers = pd.read_pickle('pickles/customers_second_iteration.pkl')

In [8]:
from utils.add_uniclass_features import AddUniclassFeatures

add_uniclass_features = AddUniclassFeatures(articles, customers)
data_train_uniclass = add_uniclass_features.fit_transform(data_train_uniclass)

In [11]:
data_train_uniclass['label'].value_counts()

0    3452294
1      17160
Name: label, dtype: int64

-> Le jeu de données est déséquilibré en l'état, il faudra certainement équilibrer les données d'entraînement pour obtenir un résultat optimal.

In [10]:
data_train_uniclass.to_pickle('pickles/second_iteration_data_train_uniclass.pkl')

In [29]:
data_train_uniclass.head(5)

,customer_id,article_id,label,in_pair_list,in_repurchase_list,in_cross_list,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,...,repurchases_customer,repurchases_interval,age_around_15_customer,age_around_25_customer,age_around_35_customer,age_around_45_customer,age_around_55_customer,age_around_65_customer,postal_code_group,group
0,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,0915529003,1,100,100,6,Sweater,Garment Upper body,Solid,Black,...,2,16.5,0.2,0.8,0.0,0.0,0.0,0.0,0,1
1,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,0915529003,1,100,100,6,Sweater,Garment Upper body,Solid,Black,...,2,16.5,0.2,0.8,0.0,0.0,0.0,0.0,0,1
2,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,0448509014,1,100,100,12,Trousers,Garment Lower body,Solid,Blue,...,2,16.5,0.2,0.8,0.0,0.0,0.0,0.0,0,1
3,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,0448509014,1,100,100,12,Trousers,Garment Lower body,Solid,Blue,...,2,16.5,0.2,0.8,0.0,0.0,0.0,0.0,0,1
4,00250644a9628fb8c4aebabf555d3caabe44c99c74fbcf...,0885951001,1,100,100,36,Bra,Underwear,Solid,Black,...,2,3.0,0.0,0.6,0.4,0.0,0.0,0.0,4,1


### Création du dataset pour entraînement multilabel

In [29]:
from sklearn.base import TransformerMixin

class ListToMultiLabel(TransformerMixin):
    def __init__(self, articles, customers):
        self.articles = articles
        self.customers = customers
        
        self.articles['article_id'].astype("int32")
        return
    
    def fit(self, dataset):
        return self
        
    def transform(self, dataset):
        
        dataset = dataset.merge(customers, on = 'customer_id', how = 'left')

        for i in range(0, 50):
            print(f"\r Process item n°{i + 1}", end = "")
            
            dataset[f"article_id_{i}"] = dataset['shortlist'].apply(lambda x: x[i])
            dataset[f"{i}_label"] = dataset.apply(lambda x: 1 if x[f"article_id_{i}"] in x['purchased_list'] else 0, axis = 1)
            dataset[f"{i}_in_pair_list"] = dataset.apply(lambda x: 1 if x[f"article_id_{i}"] in x['pair_list'] else 0, axis = 1)
            dataset[f"{i}_in_repurchase_list"] = dataset.apply(lambda x: 1 if x[f"article_id_{i}"] in x['repurchase_list'] else 0, axis = 1)
            dataset[f"{i}_in_cross_list"] = dataset.apply(lambda x: 1 if x[f"article_id_{i}"] in x['cross_list'] else 0, axis = 1)
            
            dataset[f"article_id_{i}"].astype('int32')
            
            dataset = dataset.merge(articles, left_on = f"article_id_{i}", right_on = 'article_id', how = 'left', suffixes = ('', f'_{i}'))
            
            dataset[f"article_id_{i}"] = "0" + dataset[f"article_id_{i}"].astype(str)
        return dataset
            
            
            

In [26]:
data_train = pd.read_pickle('pickles/second_iteration_data_train_lists.pkl')

In [27]:
# Nettoyage.
data_train['shortlist_length'] = data_train['shortlist'].apply(lambda x: len(x))
data_train = data_train[data_train['shortlist_length'] >= 50]

In [30]:
list_to_multilabel = ListToMultiLabel(articles, customers)

data_train_multiclass = list_to_multilabel.fit_transform(data_train)

 Process item n°49

In [34]:
data_train_multiclass.to_pickle('pickles/second_iteration_data_train_multilabel.pkl')
data_train_multiclass

,customer_id,purchased_list,cross_list,pair_list,repurchase_list,shortlist,shortlist_length,FN,Active,club_member_status,...,average_price_49,has_image_49,age_around_15_49,age_around_25_49,age_around_35_49,age_around_45_49,age_around_55_49,age_around_65_49,repurchases_49,repurchase_interval_49
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[0624486001],"[0915526001, 0915526001, 0919365008, 091936500...",[0751471043],"[0748269009, 0881919001, 0895610005, 090222900...","[0748269009, 0881919001, 0895610005, 090222900...",51,0,0,ACTIVE,...,0.029988,True,0.107692,0.483745,0.206077,0.100236,0.091074,0.010668,0.028302,0.062772
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[0827487003],"[0919365008, 0919365008, 0706016001, 070601600...","[0751471043, 0554772003, 0858052008, 083211400...","[0652924004, 0797892001, 0900176002, 044851500...","[0652924004, 0797892001, 0900176002, 044851500...",51,1,1,ACTIVE,...,0.033135,True,0.120074,0.387941,0.150503,0.169277,0.133603,0.035882,0.027206,0.038419
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[0757926001, 0788575004, 0640021019]","[0706016001, 0706016001, 0448509014, 044850901...","[0862482007, 0875350003, 0832453003, 082793500...","[0111593001, 0658030020, 0863001001]","[0111593001, 0658030020, 0863001001, 086248200...",51,0,0,ACTIVE,...,0.016238,True,0.185333,0.473647,0.109055,0.129768,0.084078,0.011686,0.020392,0.955490
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[0874110016],"[0919365008, 0706016001, 0797988002, 044850901...",[0751471043],"[0714790020, 0448509018, 0658298001, 079479300...","[0714790020, 0448509018, 0658298001, 079479300...",51,1,1,ACTIVE,...,0.016454,True,0.154822,0.357022,0.166639,0.225746,0.083249,0.011675,0.013536,0.006768
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[0903762001, 0879189005, 0799365027, 048663900...","[0915526001, 0915526001, 0919365008, 091936500...","[0751471043, 0751471043, 0562245050, 087253700...","[0687704022, 0759465001, 0887949001, 085912500...","[0687704022, 0759465001, 0887949001, 085912500...",51,0,0,ACTIVE,...,0.024661,True,0.091288,0.393615,0.126227,0.141414,0.204004,0.039610,0.063853,0.073593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68726,fffa67737587e52ff1afa9c7c6490b5eb7acbc439fe82b...,"[0874816003, 0911870004]","[0915526001, 0915526001, 0751471001, 075147100...",,,"[0915526001, 0915526001, 0751471001, 075147100...",50,0,0,ACTIVE,...,0.024420,True,0.109207,0.333858,0.125267,0.178558,0.193043,0.049066,0.073981,5.057137
68727,fffa7d7799eb390a76308454cbdd76e473d65b1497fbe4...,"[0849886010, 0861803014]","[0852584001, 0852584001, 0751471043, 075147104...",,,"[0852584001, 0852584001, 0751471043, 075147104...",50,0,0,ACTIVE,...,0.024661,True,0.091288,0.393615,0.126227,0.141414,0.204004,0.039610,0.063853,0.073593
68728,fffae8eb3a282d8c43c77dd2ca0621703b71e90904dfde...,"[0918200001, 0894320003, 0715624050, 082615000...","[0919365008, 0915526001, 0706016001, 044850901...",,,"[0919365008, 0915526001, 0706016001, 044850901...",50,0,0,ACTIVE,...,0.047949,True,0.083137,0.465345,0.179608,0.117143,0.122762,0.027983,0.071360,12.871694
68729,fffd870c6324ad3bda24e4d6aeae221c199479086bfdfd...,"[0750423010, 0761269001]","[0751471043, 0751471001, 0915529003, 086359500...",,,"[0751471043, 0751471001, 0915529003, 086359500...",50,1,1,ACTIVE,...,0.024355,True,0.079241,0.334526,0.149624,0.156121,0.193496,0.071355,0.036585,5.630578
